In [1]:
import polars as pl
# import polars_ols as pls
import pandas as pd
import numpy as np
import rolch

from datetime import datetime

In [5]:
data_full = pl.read_csv('DE_prep.csv').with_columns(pl.col('Temp_DA').cast(pl.Float64))
data_full

Date,Hour,Price,Load_DA,Load_Act,Solar_DA,Solar_Act,WindOn_DA,WindOn_Act,WindOff_DA,WindOff_Act,Temp_DA,Year
i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
20150105,0,22.34,43.17617,46.66787,0.0,0.0,10.00974,12.71664,0.19212,0.47802,null,2015
20150105,1,17.93,41.59758,44.62704,0.0,0.0,10.28754,12.5964,0.19238,0.46855,null,2015
20150105,2,15.17,40.42159,43.63534,0.0,0.0,10.41837,12.81819,0.195,0.46556,null,2015
20150105,3,16.38,40.40436,43.68339,0.0,0.0,10.6018,13.01156,0.19362,0.45402,null,2015
20150105,4,17.38,41.34813,44.96917,0.0,0.0,10.80991,13.54185,0.18762,0.45748,null,2015
…,…,…,…,…,…,…,…,…,…,…,…,…
20231001,19,130.0,45.85029,48.01121,0.01088,0.00331,7.94233,5.62464,2.78673,2.60925,18.88,2023
20231001,20,130.0,44.79807,46.51472,0.0,0.0,8.20989,6.38471,2.47612,2.09245,17.76,2023
20231001,21,115.16,42.82068,43.97374,0.0,0.0,8.36224,6.97915,2.22223,2.03959,17.0,2023


In [6]:
# cut data
data_full = data_full[52776:,:]
data_full

Date,Hour,Price,Load_DA,Load_Act,Solar_DA,Solar_Act,WindOn_DA,WindOn_Act,WindOff_DA,WindOff_Act,Temp_DA,Year
i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
20210112,0,18.96,50.84298,59.75995,0.0,0.0,33.522,32.99134,5.2638,4.71488,1.0,2021
20210112,1,15.82,48.54194,58.14762,0.0,0.0,33.86269,33.58869,5.08334,4.63254,1.16,2021
20210112,2,19.0,47.89552,56.97078,0.0,0.0,33.9029,32.90247,4.56703,3.70295,1.26,2021
20210112,3,24.14,47.92218,56.91989,0.0,0.0,33.31623,31.5854,4.16626,3.60216,1.46,2021
20210112,4,27.0,49.06122,58.05438,0.0,0.0,32.03167,30.1555,4.05303,3.23646,1.68,2021
…,…,…,…,…,…,…,…,…,…,…,…,…
20231001,19,130.0,45.85029,48.01121,0.01088,0.00331,7.94233,5.62464,2.78673,2.60925,18.88,2023
20231001,20,130.0,44.79807,46.51472,0.0,0.0,8.20989,6.38471,2.47612,2.09245,17.76,2023
20231001,21,115.16,42.82068,43.97374,0.0,0.0,8.36224,6.97915,2.22223,2.03959,17.0,2023


In [11]:
endd = [56*24,84*24]
startd = [(endd[-1]-el) for el in endd]

# no. of days to be predicted
Nd = int(len(data_full)/24 - endd[-1]/24)
print(Nd)

# select one
startd = startd[-1]
endd = endd[-1]
print(startd)
print(endd)

909
0
2016


In [12]:
def dummies(df: pl.DataFrame, date_column: str, date_format: str) -> pl.DataFrame:
    
    # prepare polars series with weekdays as numbers
    weekdays = df[date_column].cast(pl.Utf8).str.strptime(pl.Date, date_format).dt.weekday()

    # add dummy columns to dataframe
    df_with_dummies = (
    df.with_columns((weekdays==1).cast(int).alias('mon'))
    .with_columns((weekdays==2).cast(int).alias('tue'))
    .with_columns((weekdays==3).cast(int).alias('wed'))
    .with_columns((weekdays==4).cast(int).alias('thu'))
    .with_columns((weekdays==5).cast(int).alias('fri'))
    .with_columns((weekdays==6).cast(int).alias('sat'))
    .with_columns((weekdays==7).cast(int).alias('sun'))
    )

    return df_with_dummies

In [14]:
data_full = dummies(data_full,'Date',"%Y%m%d")
data_full

Date,Hour,Price,Load_DA,Load_Act,Solar_DA,Solar_Act,WindOn_DA,WindOn_Act,WindOff_DA,WindOff_Act,Temp_DA,Year,mon,tue,wed,thu,fri,sat,sun
i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64
20210112,0,18.96,50.84298,59.75995,0.0,0.0,33.522,32.99134,5.2638,4.71488,1.0,2021,0,1,0,0,0,0,0
20210112,1,15.82,48.54194,58.14762,0.0,0.0,33.86269,33.58869,5.08334,4.63254,1.16,2021,0,1,0,0,0,0,0
20210112,2,19.0,47.89552,56.97078,0.0,0.0,33.9029,32.90247,4.56703,3.70295,1.26,2021,0,1,0,0,0,0,0
20210112,3,24.14,47.92218,56.91989,0.0,0.0,33.31623,31.5854,4.16626,3.60216,1.46,2021,0,1,0,0,0,0,0
20210112,4,27.0,49.06122,58.05438,0.0,0.0,32.03167,30.1555,4.05303,3.23646,1.68,2021,0,1,0,0,0,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
20231001,19,130.0,45.85029,48.01121,0.01088,0.00331,7.94233,5.62464,2.78673,2.60925,18.88,2023,0,0,0,0,0,0,1
20231001,20,130.0,44.79807,46.51472,0.0,0.0,8.20989,6.38471,2.47612,2.09245,17.76,2023,0,0,0,0,0,0,1
20231001,21,115.16,42.82068,43.97374,0.0,0.0,8.36224,6.97915,2.22223,2.03959,17.0,2023,0,0,0,0,0,0,1


In [15]:
def autoreg(df: pl.DataFrame, cols: list, lags: list):

    # x
    x = pl.DataFrame()
    for c in cols:
        vec = df.select(c)
        for i in range(len(lags)):
            pl_series = pl.Series(name=f'x_{c}_{lags[i]}',values=vec[(max(lags)-lags[i]):len(vec)-lags[i]])
            x = pl.concat([x, pl_series.to_frame()], how="horizontal")
    
    return x

In [ ]:
# d=1
# j=endd+24
# cal_window = data_full.slice(j-24*8,9*24) #data_full.slice(startd,endd+24+24*d) 

In [ ]:
# model: 

# load(d-1) + load(d-2) + load(d-7) + solar_da(d) + windon_da(d) + windoff_da(d) + temp_da(d)

In [102]:
# cal_window

In [106]:
def prepare_data_LEAR(cal_window: pl.DataFrame, d: int, endd: int):

    if d==0:
        x_all = np.full((endd - 7 * 24 + 24 + 24*d, 7 * 24), np.nan)
    else:
        x_all = np.full((2*24, 7 * 24), np.nan)

    dummies = cal_window[7*24:,'mon':'sun'].to_numpy()

    # prepare Load
    load = autoreg(cal_window,['Load_DA','Load_Act'],[1*24,2*24,7*24])
    # change the last 14 values of x_Load_Act_24 column to 
    query = (load.with_columns(
        pl.when(pl.arange(0, load.height) >= (load.height - 13))
        .then(pl.col("x_Load_DA_24"))
        .otherwise(pl.col("x_Load_Act_24"))
        .alias("x_Load_Act_24")
    ).drop([x for x in load.columns if 'DA' in x])).lazy()
    load = query.collect()

    # prepare other columns
    fundamentals = autoreg(cal_window,['Solar_DA','WindOn_DA','WindOff_DA','Temp_DA'],[0])[168:,:]

    lags_all = np.concatenate((load.to_numpy(),fundamentals.to_numpy()),axis=1)

    for i in range(0,len(lags_all),24):
        day = lags_all[i:i+24,:].copy()
        m, n = day.shape
        day_x_r = day.reshape(1, m * n, order='F')
        x_all[i:i+24, :] = np.tile(day_x_r, (24, 1))

    x_all = np.concatenate((cal_window[7*24:,'Load_Act'].to_numpy().reshape(-1, 1),x_all,dummies),axis=1)

    x = x_all[:-24,:]
    x_fut = x_all[-24:,1:]
    
    return x,x_fut

In [ ]:
# x,x_fut = prepare_data_LEAR(cal_window,1,endd)

In [113]:
# ROLCH estimation

all_results_mean = pl.Series(dtype=pl.Float64)
all_results_std = pl.Series(dtype=pl.Float64)

equation = {
    0: "all",
    1: "all",
}

hours_ols = {}

j=endd

for d in range(0,Nd):
    arr_temp_mean = np.zeros(24)
    arr_temp_std = np.zeros(24)
    if d==0:
        data_roll = data_full.slice(startd,endd+24) # data_full.slice(startd+24*d,endd+24)
    else:
        data_roll = data_full.slice(j-24*8,9*24)

    # prepare data
    x,x_fut = prepare_data_LEAR(cal_window=data_roll,d=d,endd=endd)

    # loop over hours
    for h in range(0,24):

        # get subsets for hour h
        data_h = x[h::24]
        data_h_fut = x_fut[h::24]

        y = data_h[:,0]
        X = data_h[:,1:]

        if d==0: # start the estimator if d=0
            online_gamlss_lasso = rolch.OnlineGamlss(
                distribution=rolch.DistributionGamma(),
                method="lasso",
                scale_inputs=True,
                equation=equation,
                fit_intercept=True,
                forget=1/endd,
                estimation_kwargs={"ic": {i: "aic" for i in range(rolch.DistributionGamma().n_params)}},
                max_it_outer = 5,
                max_it_inner = 10
            )

            online_gamlss_lasso.fit(X, y)

            hours_ols[h] = online_gamlss_lasso

        else: # update the estimator if d!=0
            X_update = X[[-1],:]
            y_update = y[[-1]]

            online_gamlss_lasso = hours_ols[h]
            online_gamlss_lasso.update(X_update,y_update)
            hours_ols[h] = online_gamlss_lasso

        prog = online_gamlss_lasso.predict(
            X=data_h_fut
        )

        arr_temp_mean[h] = prog[0][0]
        arr_temp_std[h] = prog[0][1]

    all_results_mean = pl.concat(
        [all_results_mean,pl.Series(np.array(arr_temp_mean))],how='vertical'
    )

    all_results_std = pl.concat(
        [all_results_std,pl.Series(np.array(arr_temp_std))],how='vertical'
    )

    j=j+24 # for the loop over days

    if d % 50 == 0:
        print(d)

c:\Users\Katarzyna Chęć\Documents\PhD research\2. GAMLSS for load\ROLCH load\.venv\lib\site-packages\rolch\scaler.py:62: RuntimeWarning: invalid value encountered in divide
  out[:, self.to_scale] = (x[:, self.to_scale] - self.m) / np.sqrt(self.v)


SystemError: CPUDispatcher(<function online_coordinate_descent_path at 0x0000026526695AB0>) returned a result with an exception set

In [ ]:
q_results = data_full.select('Date','Hour','Load_Act')[(startd+endd):].with_columns(
    all_results_mean.alias('prediction_mean'),
    all_results_std.alias('prediction_std')).lazy()
results = q_results.collect()

In [ ]:
results.write_csv(f'DE_lasso_gamlss_load_{endd}.csv')